# function calling
Function Calling은 AI 모델이 실시간 데이터 조회나 외부 서비스 조작을 위해 필요한 '도구'를 스스로 선택하고 실행 인자를 만들어주는 기술이다.   
모델이 함수를 직접 실행하는 대신 실행에 필요한 데이터를 구조화된 JSON 형태로 짜주면, 개발자의 코드가 이를 처리해 최신 정보를 답변에 반영할 수 있다.

### 공식문서 예시
https://platform.openai.com/docs/guides/function-calling#function-tool-example

In [11]:
import os
import requests
import json
from dotenv import load_dotenv
from pprint import pprint

from openai import OpenAI
import json

load_dotenv()

TMDB_API_KEY = os.getenv('TMDB_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

model = 'gpt-4o-mini'


In [3]:
import json

# 1. 모델이 사용할 수 있는 도구(함수) 리스트 정의
tools = [
    {
        "type": "function",
        "name": "get_horoscope",
        "description": "특정 별자리의 오늘 운세를 가져옵니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "sign": {
                    "type": "string",
                    "description": "황소자리(Taurus) 또는 물병자리(Aquarius)와 같은 별자리 이름",
                },
            },
            "required": ["sign"],
        },
    },
]

def get_horoscope(sign):
    # 실제 환경에서는 API 호출 등이 일어나겠지만, 여기서는 예시 문구를 반환합니다.
    return f"{sign}: 다음 주 화요일에 아기 수달과 친구가 될 운명입니다."

# 대화 기록을 저장할 리스트 생성 (지속적으로 업데이트됨)
input_list = [
    {"role": "user", "content": "내 운세가 뭐야? 나는 물병자리(Aquarius)야."}
]

# 2. 정의된 도구와 함께 모델에 요청 전달
response = client.responses.create(
    model=model,
    tools=tools,
    input=input_list,
)

# 모델의 응답(함수 호출 요청 등)을 입력 리스트에 추가
input_list += response.output

for item in response.output:
    if item.type == "function_call":
        if item.name == "get_horoscope":
            # 3. get_horoscope 함수 로직 실행
            # 인수(arguments)를 파싱하여 실제 함수 호출
            args = json.loads(item.arguments) # 보통 dict 형태이거나 json.loads가 필요할 수 있음
            horoscope = get_horoscope(args.get("sign"))
            
            # 4. 함수 실행 결과를 모델에 다시 전달하기 위해 리스트에 추가
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps({
                  "horoscope": horoscope
                })
            })

print("최종 입력 리스트(컨텍스트):")
print(input_list)

# 모델에게 함수 결과값이 포함된 전체 컨텍스트를 전달하여 최종 답변 유도
response = client.responses.create(
    model=model,
    instructions="도구에 의해 생성된 운세 내용만 사용하여 답변해줘.",
    tools=tools,
    input=input_list,
)

# 5. 모델이 함수 결과를 바탕으로 최종 답변을 출력
print("최종 출력 결과:")
print(response.model_dump_json(indent=2))
print("\n최종 답변 문구: " + response.output_text)

최종 입력 리스트(컨텍스트):
[{'role': 'user', 'content': '내 운세가 뭐야? 나는 물병자리(Aquarius)야.'}, ResponseFunctionToolCall(arguments='{"sign":"Aquarius"}', call_id='call_2Lu7JmScDkAtrYRn0iGDXcOa', name='get_horoscope', type='function_call', id='fc_06a012c5dac712840069717687df1c81949fe8e7082d43997f', status='completed'), {'type': 'function_call_output', 'call_id': 'call_2Lu7JmScDkAtrYRn0iGDXcOa', 'output': '{"horoscope": "Aquarius: \\ub2e4\\uc74c \\uc8fc \\ud654\\uc694\\uc77c\\uc5d0 \\uc544\\uae30 \\uc218\\ub2ec\\uacfc \\uce5c\\uad6c\\uac00 \\ub420 \\uc6b4\\uba85\\uc785\\ub2c8\\ub2e4."}'}]
최종 출력 결과:
{
  "id": "resp_06a012c5dac71284006971768897cc8194961873fb2415e913",
  "created_at": 1769043592.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "도구에 의해 생성된 운세 내용만 사용하여 답변해줘.",
  "metadata": {},
  "model": "gpt-4o-mini-2024-07-18",
  "object": "response",
  "output": [
    {
      "id": "msg_06a012c5dac712840069717689712881948f38d60633e194a7",
      "content": [
        {
          "annotat

### 함수 정의
LLM이 호출할 함수를 정의한다.

In [12]:
def get_now_playing_movies(region : str = 'KR'):
    """TMDB API를 호출하여 현재 상영 중인 영화 목록을 가져옵니다."""

    print(f'{region}의 영화를 가져옵니다.')

    url = "https://api.themoviedb.org/3/movie/now_playing"
    
    params = {
        'language' : 'ko-kr',
        'region' : region
    }
    
    headers = {
        'Authorization' : f'Bearer {TMDB_API_KEY}'
    }
    
    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()    
        return response.json()['results']
    except Exception as e:
        print(e)

pprint(get_now_playing_movies())

KR의 영화를 가져옵니다.
[{'adult': False,
  'backdrop_path': '/u8DU5fkLoM5tTRukzPC31oGPxaQ.jpg',
  'genre_ids': [878, 12, 14],
  'id': 83533,
  'original_language': 'en',
  'original_title': 'Avatar: Fire and Ash',
  'overview': '인간들과의 전쟁으로 첫째 아들 ‘네테이얌’을 잃은 후, ‘제이크’와 ‘네이티리’는 깊은 슬픔에 빠진다. 상실에 '
              "빠진 이들 앞에 '바랑'이 이끄는 재의 부족이 등장하면서, 판도라는 더욱 큰 위험에 빠지게 되고, ‘설리’ 가족은 "
              '선택의 기로에 서게 되는데…',
  'popularity': 264.0534,
  'poster_path': '/l18o0AK18KS118tWeROOKYkF0ng.jpg',
  'release_date': '2025-12-17',
  'title': '아바타: 불과 재',
  'video': False,
  'vote_average': 7.348,
  'vote_count': 1528},
 {'adult': False,
  'backdrop_path': '/sK3z0Naed3H1Wuh7a21YRVMxYqt.jpg',
  'genre_ids': [9648, 53],
  'id': 1368166,
  'original_language': 'en',
  'original_title': 'The Housemaid',
  'overview': '과거를 숨긴 채 완벽한 저택의 가정부로 들어간 밀리가 이상한 행동을 보이는 아내 니나와 그런 아내를 이해하는 '
              '모범적인 남편 앤드루와 얽히며 닫힌 문 뒤의 거짓과 마주하게 되는 월드 베스트셀러 원작의 고자극 미스터리 스릴러',
  'popularity': 187.8712,
  'poster_path': '/fUDX16A4fJXmAu

### 함수 스키마 정의
LLM이 함수를 이해할 수 있도록 스키마를 정의한다.

In [7]:
tools = [
    {
        "type": "function",
        "name": "get_now_playing_movies",
        "description": "현재 극장에서 상영 중인 영화 목록을 TMDB에서 가져옵니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "region": {
                    "type": "string",
                    "description": "region, ISO-3166-1 code",
                },

            }
        }
    }
]

In [13]:

def run_conversation(user_prompt):
    
    input_list = [{"role": "user", "content": user_prompt}]

    # 1. LLM에게 1차 요청을 한다.
    response = client.responses.create(
        model=model,
        input=input_list,
        tools=tools,
    )
    input_list += response.output
    
    # 2. LLM의 응답에서 "함수 실행"에 대한 응답이 있다면 이를 실행한다.
    for item in response.output:
        if item.type == "function_call":
            if item.name == 'get_now_playing_movies':

                result = get_now_playing_movies(**json.loads(item.arguments))
                
                # 3. 함수 실행 결과를 input에 담는다.
                input_list.append({
                    "type": "function_call_output",
                    "call_id": item.call_id,
                    "output": json.dumps({
                    "result": result
                    })
                })
    # 4. 다시 한번 LLM에게 요청한다.
    response = client.responses.create(
        model=model,
        instructions="Respond only with a movies generated by a tool.",
        tools=tools,
        input=input_list,
    )

    return response.output_text
    

# 실행 예시
print(run_conversation("요즘 미국에서 볼만한 영화가 뭐가 있어? 제목만 한국어로 번역해서 알려줘"))

US의 영화를 가져옵니다.
요즘 미국에서 상영 중인 영화는 다음과 같습니다:

1. 아바타: 불과 재
2. 하우스메이드
3. 살레 웨일
4. 28년 후: 뼈의 사원
5. 그린란드 2
6. 마르티 슈프림
7. 스폰지밥 무비: 스퀘어팬츠를 찾아서
8. 아나콘다
9. 어마저지지 않아
10. 프라이메이트
11. 송 송 블루
12. 메르시
13. 반지의 제왕: 반지 원정대
14. 반지의 제왕: 왕의 귀환
15. 리턴 투 사일런트 힐
16. 반지의 제왕: 두 개의 탑
17. 찌빠 찌빠칠라
18. 핫싱이 형
19. 다비드
20. 마다가스카르


In [4]:
import requests
from pprint import pprint

def get_news(query):
    """query에 대한 뉴스 검색 결과를 가져옵니다."""

    url = "https://google.serper.dev/news"
    
    payload = {
        "q": query,
        }
    headers = {
    'X-API-KEY': 'b0ce912249e374614d40339216224abdf2da8e75',
    'Content-Type': 'application/json'
    }

    
    try:
        response = requests.request("POST", url, headers=headers, json=payload)
        response.raise_for_status()    
        return response.json()
    except Exception as e:
        print(e)

pprint(get_news('openai'))

{'credits': 1,
 'news': [{'date': '2 hours ago',
           'imageUrl': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRGydFMwj9ggafrje6NsNw_ENtczFaBmpgYMjZGhPHObLBsfn0DOloiTFM&usqp=CAI&s',
           'link': 'https://www.theinformation.com/articles/openai-names-former-tml-staffer-zoph-oversee-enterprise-push',
           'snippet': 'Barret Zoph, the Thinking Machines Lab cofounder who '
                      'rejoined OpenAI in a swirl of controversy last week, '
                      "will lead the company's push to...",
           'source': 'The Information',
           'title': 'OpenAI Names Former TML Staffer Zoph to Oversee '
                    'Enterprise Push'},
          {'date': '1 day ago',
           'imageUrl': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTNcoHiZ8PIuJtZc3qMzAKlf0GmiD2T4FNJCtB3-KIHt1OrfZvDS2MkoGo&usqp=CAI&s',
           'link': 'https://www.reuters.com/technology/openai-rolls-out-age-prediction-chatgpt-2026-01-20/',
           'source': '